In [ ]:
%pip install git+https://github.com/Open-Dataplatform/utils-databricks.git@v0.2.1

In [ ]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from custom_utils.dp_storage import connector, reader, writer
from custom_utils import adf

# Transformation

## Setup

In [ ]:
mount_point = connector.mount(dbutils)

In [ ]:
dbutils.widgets.removeAll()

# Get the parameters from adf. When developing, insert values in the text widgets.
source_config = adf.get_parameter_json('SourceConfig')
destination_config = adf.get_parameter_json('DestinationConfig')
source_folder_path = adf.get_parameter('SourceFolderPath')  # Remember that it has the format "<container>/<directory>"
source_filename = adf.get_parameter('SourceFileName')

## Read
Reads data from storage

In [ ]:
source_file_path = reader.get_path_to_triggering_file(
    mount_point,
    source_folder_path,
    source_filename,
    config_for_triggered_dataset=source_config['TODO: dataset_identifier']
)

df = spark.read.parquet(source_file_path)
# Rewrite the line above, if your file is not parquet. Example for csv:
# df = spark.read.option("delimiter", ",").csv(source_path, header=True)

In [ ]:
df.show(3)

## Transform

Transform the data here. Follow this style guide: https://github.com/palantir/pyspark-style-guide

## Merge and upload

In [ ]:
# TODO: Update the following parameters
timestamp_column = 'NAME_OF_TIMESTAMP_COLUMN'
index_columns = [timestamp_column, 'ANOTHER_KEY_COLUMN']
time_resolution_egress = 'month'  # 'hour', 'month', 'day', or 'hour'
egress_identifier = destination_config['YOUR_EGRESS_DATASET_IDENTIFIER']['dataset']

# Merge data into the existing egress data.
# If the timestamp column is a string column, add the format in the parameter time_format (example: "yyyy-MM-dd'T'HH:mm:ss'Z'")
writer.merge_and_upload(
    df,
    egress_identifier,
    timestamp_column,
    index_columns,
    time_resolution_egress,
    mount_point,
    spark,
    time_format=''
)

In [ ]:
# Always keep this at the end of your notebook
connector.unmount_if_prod(mount_point, dbutils)